In [29]:
import pandas as pd
import matplotlib.pyplot as plt

In [30]:
df_bitcoin = pd.read_csv("../data/bitcoin_features.csv", index_col=0)
df_rates = pd.read_csv("../data/FEDFUNDS.csv")

In [31]:
df_bitcoin.shape

(4378, 21)

In [32]:
df_rates.shape

(121, 2)

In [33]:
df_bitcoin.dtypes

date                         object
active_addresses            float64
tweets                      float64
top_100_percent             float64
difficulty                  float64
transactions                float64
av_transaction_size         float64
market_cap                  float64
confirmation_time           float64
median_transaction_value    float64
send_usd                    float64
google_trends               float64
block_size                  float64
mining_profitability        float64
hashrate                    float64
sent_addresses              float64
median_transaction_size     float64
fee_reward                  float64
av_transaction_value        float64
full_name                    object
coin                         object
dtype: object

In [34]:
df_rates.dtypes

observation_date     object
FEDFUNDS            float64
dtype: object

In [35]:
df_bitcoin = df_bitcoin.dropna()
df_bitcoin.isna().sum()

date                        0
active_addresses            0
tweets                      0
top_100_percent             0
difficulty                  0
transactions                0
av_transaction_size         0
market_cap                  0
confirmation_time           0
median_transaction_value    0
send_usd                    0
google_trends               0
block_size                  0
mining_profitability        0
hashrate                    0
sent_addresses              0
median_transaction_size     0
fee_reward                  0
av_transaction_value        0
full_name                   0
coin                        0
dtype: int64

In [36]:
df_bitcoin.head()

,date,active_addresses,tweets,top_100_percent,difficulty,transactions,av_transaction_size,market_cap,confirmation_time,median_transaction_value,...,google_trends,block_size,mining_profitability,hashrate,sent_addresses,median_transaction_size,fee_reward,av_transaction_value,full_name,coin
